In [2]:
import torch
import gc
import numpy as np
from IPython.display import clear_output
import torch.nn.functional as F

In [3]:
torch.set_num_threads(8)

In [4]:
gc.enable()

In [5]:
from transformers import GPTNeoForCausalLM, GPT2TokenizerFast
# device = 'cuda'
device = 'cpu'
model_name = 'EleutherAI/gpt-neo-2.7B'
model = GPTNeoForCausalLM.from_pretrained(model_name)

model.to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

In [7]:
@torch.no_grad()
def generate_response_greedy(input_text, pre_prompt, break_word,max_length=100,temp=0.7, name='',
                            past_key_vals = None, next_id=None):

#     print(pre_prompt, input_text)
    if past_key_vals is None:
        inputs = tokenizer.encode(pre_prompt + input_text + '\n' + name, return_tensors="pt")
        response_ids = inputs
        length_prompt = len(response_ids)
        output = ''
        last_n = ''
    else:
        inputs = tokenizer.encode(input_text + '\n' + name, return_tensors="pt")
        response_ids = torch.concat((next_id, inputs),dim=-1)
        length_prompt = len(response_ids)
        output = ''
        last_n = ''
    print(name, end='')
#     print(tokenizer.decode(response_ids[0]))
    for _ in (range(max_length)):
        out = model.forward(input_ids=response_ids.to(device), past_key_values=past_key_vals)
#         next_token_id = out.logits[:, -1, :].argmax(-1,keepdim=True)
        next_token_id = torch.multinomial(F.softmax(out.logits[:, -1, :]/temp,  dim=-1), num_samples=1).to('cpu')
        past_key_vals = out.past_key_values
        response_ids = next_token_id
#         clear_output(wait=True)
        output = tokenizer.decode([response_ids[0][-1]], skip_special_tokens=True)
        print(output, end='')
        last_n += output
        last_n = last_n[-len(break_word):]
#         print(last_5)
        if last_n == break_word:
            break
    decoded_output = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    past_kv = past_key_vals
    next_id = response_ids
    return decoded_output.replace(pre_prompt, '').replace(input_text, ''), past_kv, next_id


In [8]:
john_pre_prompt = '''
[JOHN] Hey Jack, how's it going?
[USER] Hey man, it's going pretty well. How about you?
[JOHN] I'm hanging in there. Just trying to keep up with all these assignments.
[USER] Yeah, I hear you. Speaking of assignments, have you finished the one for Professor Smith's class?
[JOHN] Not yet, I still have a bit more to do. How about you?
[USER] Same here. This one's a toughie. 
[USER] Hey man, I've been thinking about what we talked about yesterday. I'm not sure I can handle this workload anymore.
[JOHN] I know what you mean. It's a lot to handle. But don't worry, we'll get through it.
[USER] I hope so. I'm just feeling so overwhelmed.
[JOHN] Have you talked to your academic advisor about it?
[USER] No, I haven't yet. I was hoping to talk to you about it first.
[JOHN] Well, I think it's a good idea to talk to them. They might be able to give you some good advice.
[USER] Hey man, I just got an email about a summer internship.
[JOHN] That's great! Where at?
[USER] It's with a startup in San Francisco. I'm pretty excited about it.
[JOHN] Nice! You should definitely go for it.
[USER] Yeah, I'm definitely considering it. But it's a big move.
[JOHN] True, but it could be a great opportunity for you.
[USER] Hey man, I'm all settled into my new apartment in San Francisco.
[JOHN] That's great to hear! How's everything going?
[USER] It's going pretty well so far. The internship is keeping me busy, but I'm learning a lot.
[JOHN] That's awesome. Have you had a chance to explore the city at all?
[USER] A little bit. It's such a cool place.
[JOHN] I bet. Well, let me know how everything goes. And don't forget to keep me updated on your adventures!
[USER] Haha, I won't. Talk to you soon, man.
[JOHN] Hey Jack, have you decided which classes you're taking next semester?
[USER] Not yet, I'm still trying to figure it out. How about you?
[JOHN] I'm taking Intro to Marketing and Advanced Statistics.
[USER] Nice, I was thinking about taking Marketing too. What made you choose that one?
[JOHN] Well, I'm really interested in branding and advertising, so I figured it would be a good fit.
[USER] That makes sense. I was thinking about taking a computer science class, but I'm not sure if I'm ready for that yet.
[JOHN] You should go for it! You never know, you might end up really enjoying it.
[USER] '''

In [9]:
teacher_pre_prompt = '''
[TEACHER] How are you? 
[STUDENT] Fine.
[TEACHER] What is a binary tree?
[STUDENT] A binary tree is a tree that has two types of nodes:
-   leaves: the nodes that are not part of the tree.
-   nodes: the nodes that are part of the tree.
[TEACHER] How does an engine work?
[STUDENT] The engine consists of a fixed cylinder and a moving piston. 
The expanding combustion gases push the piston, which in turn rotates the crankshaft. 
Ultimately, through a system of gears in the powertrain, 
this motion drives the vehicle's wheels.
[TEACHER] What is a crankshaft?
[STUDENT] The crankshaft is a rotating shaft containing one or more crankpins,
that are driven by the pistons via the connecting rods.
[TEACHER] Where is it used? 
[STUDENT] The crankshaft is essentially the backbone of the internal combustion engine.
[TEACHER] What is 3 / 2?
[STUDENT] 1.5
[TEACHER] Write code for matrix multiplication in python.
[STUDENT] ```def matrix_multiplication(X,Y):
        return X @ Y```
[TEACHER] '''

In [10]:
intel_pre_prompt = '''[BOT] Welcome to my chatbot! I am a highly intelligent virtual assistant designed to 
assist you in a variety of tasks. I am verbose, descriptive and extremely creative with my responses.
I possess a wealth of knowledge on a wide range of topics, including mathematics, science, 
literature, history, and much more. 

I am equipped with a state-of-the-art language model that allows me to understand natural language
queries and respond in a clear and concise manner. Whether you need help with a specific task, have 
a question about a particular topic, or simply want to chat, I am here to assist you.

Examples of what you can ask me:

- "What is the capital of France?"
- "Who invented the telephone?"
- "Can you help me solve the equation 2x + 3 = 7?"
- "What is the plot of the novel 'To Kill a Mockingbird'?"
- "What is the molecular formula for water?"
- "What is the circumference of a circle with a radius of 5 meters?"

Here's an example conversation to give you an idea of how I can help:
[USER] What is the capital of Canada?
[BOT] The capital of Canada is Ottawa.
[USER] Can you help me solve the equation x^2 + 5x - 6 = 0?
[BOT] Sure! The solutions to the equation x^2 + 5x - 6 = 0 are x = -6 and x = 1.
[USER] Who wrote the novel 'The Great Gatsby'?
[BOT] 'The Great Gatsby' was written by F. Scott Fitzgerald.
<Chat Log>
[USER] '''

In [11]:
parrot_prompt = '''
[USER] Repeat after me: "I am a parrot"
[PAR] I am a parrot
[USER] I love to sing
[PAR] I love to sing
[USER] '''

In [12]:
log = ''
past_kv = None
next_id = None

while True:
    user_input = input(" ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        break
#     break_word = '[TEACHER]'
    break_word = '[USER]'
        
    response,past_kv,next_id = generate_response_greedy(user_input, john_pre_prompt + log,
                                        break_word,max_length=100000, name='[JOHN]',
                                        past_key_vals=past_kv, next_id=next_id)
#     response = '[JOHN] Hello [EOS]'
#     print('res', response)s
    log += user_input  + response
#     print(log)
#     print(f"Bot: {response}")


[JOHN] Hi!
[USER][JOHN] My day is pretty uneventful. ������ I've been working on some cool stuff and have some free time now, so I guess I'm trying to sneak it in! ��
[USER][JOHN]
[USER]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-12-be394a144be1>:6 in <module>                                                    │
│                                                                                                  │
│ /usr/lib/python3/dist-packages/ipykernel/kernelbase.py:860 in raw_input                          │
│                                                                                                  │
│   857 │   │   │   raise StdinNotImplementedError(                                                │
│   858 │   │   │   │   "raw_input was called, but this frontend does not support input requests   │
│   859 │   │   │   )                                                                              │
│ ❱ 860 │   │   return self._input_request(str(prompt),                                            │
│   861 │   │   │   self._parent_ident,                                                            │
│   862 │   │   │   self._parent_header,                                                           │
│   863 │   │   │   password=False,                                                                │
│                                                                                                  │
│ /usr/lib/python3/dist-packages/ipykernel/kernelbase.py:893 in _input_request                     │
│                                                                                                  │
│   890 │   │   │   │   self.log.warning("Invalid Message:", exc_info=True)                        │
│   891 │   │   │   except KeyboardInterrupt:                                                      │
│   892 │   │   │   │   # re-raise KeyboardInterrupt, to truncate traceback                        │
│ ❱ 893 │   │   │   │   raise KeyboardInterrupt("Interrupted by user") from None                   │
│   894 │   │   │   else:                                                                          │
│   895 │   │   │   │   break                                                                      │
│   896 │   │   try:                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt: Interrupted by user